In [53]:
# To run docker image: 
#   docker run -p 8000:8000 allennlp/allennlp:v0.4.1 python -m allennlp.run serve

In [63]:
import requests
import json
from pprint import pprint

def resolve_co_referents(txt):

    #url = "http://demo.allennlp.org/predict/coreference-resolution"
    url ="http://localhost:8000/predict/coreference-resolution"

    doc = {"document":txt}

    headers = {
        'content-type': "application/json",
        'DNT': "1"
    }

    response = requests.request("POST", url, data=json.dumps(doc), headers=headers)
    response.status_code
    return response.json()

In [78]:
txt = "We 're not going to skimp on quality , but we are very focused to make next year . The only problem is that some of the fabrics are wearing out - since I was a newbie I skimped on some of the fabric and the poor quality ones are developing holes."
txt = "Friday's 190 - point plunge in stocks does not come atop the climate of anxiety that dominated financial markets just prior to their 1987 October crash , and mechanisms have been put in place to keep markets more orderly . Still , the lesson is about the same : On Friday the 13th , the market was spooked by Washington. The consensus along the street seems to be that the plunge was triggered by the financing problems of the UAL takeover, and it's certainly true the rout began immediately after the UAL trading halt. Still, the consensus seems almost as wide that one faltering bid is no reason to write down the value of all U.S. business. This observation leads us to another piece of news moving on the Dow Jones ticker shortly before the downturn: the success of Senate Democrats in stalling the capital gains tax cut. The real value of all shares , after all , is directly impacted by the tax on any profits (all the more so given the limits on deductions for losses that show gains are not  'ordinary income'). And market expectations clearly have been raised by the capital gains victory in the House last month. An hour before Friday 's plunge , that provision was stripped from the tax bill, leaving it with $5.4 billion in tax increases without a capital gains cut . There is a great deal to be said , to be sure , for stripping the garbage out of the reconciliation bill . It would be a good thing if Congress started to decide issues one - by - one on their individual merits without trickery. For one thing, no one doubts that the capital gains cut would pass on an up-or-down vote . Since Senate leaders have so far fogged it up with procedural smokescreens , promises of a cleaner bill are suspect . Especially so since President Bush has been weakened by the Panama fiasco. To the extent that the UAL troubles contributed to the plunge , they are another instance of Washington 's sticky fingers. As the best opportunities for corporate restructurings are exhausted of course, at some point the market will start to reject them . But the airlines are scarcely a clear case , given anti-takeover mischief by Secretary of Transportation Skinner , who professes to believe safety will be compromised if KLM and British Airways own interests in companies that fly airplanes . Worse , Congress has started to jump on the Skinner bandwagon . James Oberstar , the Minnesota Democrat who chairs the Public Works and Transportation Committee 's aviation subcommittee , has put an anti-airline takeover bill on supersonic speed so that it would be passed in time to affect the American and United Air Lines bids . It would give Mr. Skinner up to 50 days to `` review '' any bid for 15% or more of the voting stock of any U.S. carrier with revenues of $1 billion or more . So the UAL deal has problems , and the market loses 190 points . Congratulations , Mr. Secretary and Mr. Congressman . In the 1987 crash , remember , the market was shaken by a Danny Rostenkowski proposal to tax takeovers out of existance . Even more important , in our view , was the Treasury 's threat to thrash the dollar . The Treasury is doing the same thing today ; thankfully , the dollar is not under 1987 - style pressure . Also , traders are in better shape today than in 1987 to survive selling binges . They are better capitalized . They are in less danger of losing liquidity simply because of tape lags and clearing and settlement delays . The Fed promises any needed liquidity . The Big Board 's liaison with the Chicago Board of Trade has improved; it will be interesting to learn if  'circuit breakers' prove to be a good idea . In any event , some traders see stocks as underpriced today , unlike 1987. There is nothing wrong with the market that can't be cured by a little coherence and common sense in Washington . But on the bearish side , that may be too much to expect. First Chicago Corp. posted a third - quarter loss of $23.3 million after joining other big banks in further adding to its reserves for losses on foreign loans . The parent company of First National Bank of Chicago, with $48 billion in assets , said it set aside $200 million to absorb losses on loans and investments in financially troubled countries. The addition, on top of two big 1987 additions to foreign-loan reserves , brings the reserve to a level equaling 79% of medium-term and long-term loans outstanding to troubled nations. First Chicago since 1987 has reduced its loans to such nations to $1.7 billion from $3 billion. Despite this loss, First Chicago said it doesn't need to sell stock to raise capital."
r = resolve_co_referents(txt)


In [79]:
for k,v in r.items():
    print(k.ljust(25),len(v))

antecedent_indices        150
clusters                  21
document                  893
predicted_antecedents     357
top_spans                 357


In [81]:
tokens = r["document"]
id2token = dict(enumerate(tokens))

for cl in r["clusters"]:
    print("-" * 10)
    for pair in cl:
        a,b = pair
        span = []
        for ix in range(a,b+1):
            span.append(id2token[ix])
        print(" ".join(span))

----------
dominated financial markets
their
----------
the plunge
the rout
the downturn
Friday 's plunge
the plunge
----------
UAL
UAL
UAL
UAL
----------
The consensus along the street
the consensus
----------
Friday 's
Friday 's
----------
that provision
it
----------
Congress
their
Congress
----------
the capital gains tax cut
the capital gains cut
it
----------
Senate
Senate
----------
the UAL troubles
they
----------
Washington
Washington 's
Washington
----------
the market
the market
the market
the market
----------
corporate restructurings
them
the airlines
----------
an anti - airline takeover bill on supersonic
it
It
----------
Skinner
Mr. Skinner
----------
the Treasury 's
The Treasury
----------
the dollar
the dollar
----------
today
today
today
----------
1987
1987
1987
1987
1987
----------
traders
They
They
----------
First Chicago Corp.
its
it
First Chicago
its
First Chicago
it


In [82]:
len(r["predicted_antecedents"]), len(r["top_spans"])

(357, 357)

In [84]:
tokens = r["document"]
id2token = dict(enumerate(tokens))

def span2txt(span):
    a,b = span
    tokens = []
    for ix in range(a,b+1):
        tokens.append(id2token[ix])
    return " ".join(tokens)

spans = r["top_spans"]
ants = r["predicted_antecedents"]

# for span,ant_ix in zip(spans,ants):
#     if ant_ix > -1:
#         phrase = span2txt(span)
#         reference = span2txt(spans[ant_ix])
#         print(phrase, ">>>", reference)

In [85]:
for i,sp in enumerate(spans):
    ref = ""
    if ants[i] > -1:
        ref = span2txt(spans[ants[i]+1])
    print(i, span2txt(sp).ljust(50), "\t", ants[i], ref)

0 Friday 's                                          	 -1 
1 Friday 's 190 - point plunge in stocks             	 -1 
2 190                                                	 -1 
3 stocks                                             	 -1 
4 come                                               	 -1 
5 anxiety that dominated financial markets           	 -1 
6 dominated                                          	 -1 
7 dominated financial markets                        	 -1 
8 their                                              	 0 Friday 's 190 - point plunge in stocks
9 their 1987 October crash                           	 -1 
10 1987                                               	 -1 
11 1987 October                                       	 -1 
12 October                                            	 -1 
13 mechanisms                                         	 -1 
14 put                                                	 -1 
15 place                                              	 -1 
16 keep      

335 foreign - loan reserves                            	 -1 
336 brings                                             	 -1 
337 the reserve                                        	 -1 
338 troubled                                           	 -1 
339 troubled nations                                   	 -1 
340 First Chicago                                      	 37 UAL
341 since                                              	 -1 
342 1987                                               	 114 a great deal to be said , to be sure
343 reduced                                            	 -1 
344 its                                                	 3 come
345 its loans                                          	 -1 
346 its loans to such nations                          	 -1 
347 such nations                                       	 -1 
348 this loss                                          	 -1 
349 First Chicago                                      	 8 their 1987 October crash
350 said           

In [86]:
max(ants)

142

In [87]:
r

{'antecedent_indices': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'clusters': [[[17, 19], [23, 23]],
  [[71, 72], [89, 90], [141, 142], [221, 223], [362, 363]],
  [[81, 81], [95, 95], [358, 358], [537, 537]],
  [[62, 66], [101, 102]],
  [[0, 1], [221, 222]],
  [[225, 226], [235, 235]],
  [[2